# This notebook is whrer I have trained teh Regression model to learn how the customers are reacting to the price ,the inventory changes and price changes

In [18]:
#imprting essentials
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import os



In [19]:
#GET  IN THE DATA
df = pd.read_csv("A:\\study\\projects\\EtE_ETL_Dynamic_pricing__ML\\data\\transactions.csv")

In [21]:
df.head(25)

,timestamp,product_name,price_offered,inventory_level,budget_multiplier,purchased
0,2026-01-13 12:48:04,Eggs,3.45,100,1.17,1
1,2026-01-13 12:48:04,Bread,1.18,100,1.14,1
2,2026-01-13 12:48:04,Cheese,6.50,100,0.74,0
3,2026-01-13 12:48:04,Milk,2.00,100,1.23,0
4,2026-01-13 12:48:05,Eggs,5.03,99,1.11,0
5,2026-01-13 12:48:05,Meat,7.67,100,0.85,0
6,2026-01-13 12:48:05,Eggs,5.03,99,1.30,0
7,2026-01-13 12:48:05,Milk,2.00,100,1.29,0
8,2026-01-13 12:48:05,Eggs,5.03,99,0.91,0
9,2026-01-13 12:48:05,Bread,0.90,99,0.81,0


In [22]:
#to cehck how mnay times inventory was 0 and thus people walked away
zr = df[df["inventory_level"]==0]


In [24]:
zr.head(15)

,timestamp,product_name,price_offered,inventory_level,budget_multiplier,purchased
1548,2026-01-13 12:52:05,Meat,8.00,0,1.03,0
1549,2026-01-13 12:52:05,Meat,8.00,0,0.92,0
1551,2026-01-13 12:52:06,Meat,8.00,0,0.81,0
1553,2026-01-13 12:52:06,Meat,7.28,0,0.83,0
1563,2026-01-13 12:52:07,Meat,7.28,0,0.83,0
1567,2026-01-13 12:52:08,Meat,7.28,0,0.66,0
1568,2026-01-13 12:52:08,Meat,7.28,0,1.23,0
1573,2026-01-13 12:52:09,Meat,10.25,0,0.74,0
1575,2026-01-13 12:52:09,Meat,6.82,0,0.90,0
1579,2026-01-13 12:52:10,Meat,10.80,0,1.20,0


In [23]:
zr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 1548 to 2021
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          250 non-null    object 
 1   product_name       250 non-null    object 
 2   price_offered      250 non-null    float64
 3   inventory_level    250 non-null    int64  
 4   budget_multiplier  250 non-null    float64
 5   purchased          250 non-null    int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 13.7+ KB


# experiment 
I am trying to remove the rows where the inventory was 0 because it is creating a bias in the model for meat

In [29]:
df = df[df['inventory_level'] != 0]


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1772 entries, 0 to 2015
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          1772 non-null   object 
 1   product_name       1772 non-null   object 
 2   price_offered      1772 non-null   float64
 3   inventory_level    1772 non-null   int64  
 4   budget_multiplier  1772 non-null   float64
 5   purchased          1772 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 96.9+ KB


# Implementing Features and deciding whats the target value

In [32]:
X = df[['price_offered', 'inventory_level']].astype(float)
y = df['purchased'].astype(int)

# Splitting the data

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
df.head(10)

,timestamp,product_name,price_offered,inventory_level,budget_multiplier,purchased
0,2026-01-13 12:48:04,Eggs,3.45,100,1.17,1
1,2026-01-13 12:48:04,Bread,1.18,100,1.14,1
2,2026-01-13 12:48:04,Cheese,6.50,100,0.74,0
3,2026-01-13 12:48:04,Milk,2.00,100,1.23,0
4,2026-01-13 12:48:05,Eggs,5.03,99,1.11,0
5,2026-01-13 12:48:05,Meat,7.67,100,0.85,0
6,2026-01-13 12:48:05,Eggs,5.03,99,1.30,0
7,2026-01-13 12:48:05,Milk,2.00,100,1.29,0
8,2026-01-13 12:48:05,Eggs,5.03,99,0.91,0
9,2026-01-13 12:48:05,Bread,0.90,99,0.81,0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1772 entries, 0 to 2015
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          1772 non-null   object 
 1   product_name       1772 non-null   object 
 2   price_offered      1772 non-null   float64
 3   inventory_level    1772 non-null   int64  
 4   budget_multiplier  1772 non-null   float64
 5   purchased          1772 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 96.9+ KB


# Training 

In [43]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, class_weight="balanced", random_state=42)
model.fit(x_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=5, random_state=42)

In [44]:
pred = model.predict(x_test)

# Accuracy and Eval

In [45]:
accuracy = accuracy_score(y_test, pred)

In [46]:
print(f"Accuracy is {accuracy:.2f}")

Accuracy is 0.76


In [47]:
print("Classification Report:")
print(classification_report(y_test, pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.78      0.83       262
           1       0.54      0.71      0.61        93

    accuracy                           0.76       355
   macro avg       0.71      0.75      0.72       355
weighted avg       0.79      0.76      0.77       355



# exporting the model

In [48]:


joblib.dump(model, "models/predictor3.pkl")

['models/predictor3.pkl']